### Giới thiệu
- Sử dụng mediapipe để lấy các điểm trên cơ thể để nhận diện hành động

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential

from sklearn.model_selection import train_test_split

In [ ]:
bodyswing_df = pd.read_csv("/content/drive/MyDrive/LSTM/BodySwing.txt")
handswing_df = pd.read_csv("/content/drive/MyDrive/LSTM/HandSwing.txt")
sit_df = pd.read_csv("/content/drive/MyDrive/LSTM/Sit.txt")

In [ ]:
X = []
y = []
no_of_timesteps = 10

dateset = bodyswing_df.iloc[:, 1:].values
n_sample = len(dateset)
for i in range(no_of_timesteps, n_sample):
    X.append(dateset[i-no_of_timesteps:i, :])
    y.append(0)

dateset = handswing_df.iloc[:, 1:].values
n_sample = len(dateset)
for i in range(no_of_timesteps, n_sample):
    X.append(dateset[i-no_of_timesteps:i, :])
    y.append(1)
  
dateset = sit_df.iloc[:, 1:].values
n_sample = len(dateset)
for i in range(no_of_timesteps, n_sample):
    X.append(dateset[i-no_of_timesteps:i, :])
    y.append(2)

In [ ]:
X[0]

array([[ 6.21376812e-01,  6.72498047e-01, -1.25376606e+00, ...,
         2.73322678e+00, -5.63408554e-01,  2.79548665e-04],
       [ 6.03450775e-01,  6.71223998e-01, -1.13798451e+00, ...,
         2.71726370e+00, -4.71413076e-01,  2.65539362e-04],
       [ 5.91825485e-01,  6.71208918e-01, -1.13324475e+00, ...,
         2.70469618e+00, -4.40093756e-01,  2.54157465e-04],
       ...,
       [ 5.41599751e-01,  6.74836516e-01, -1.06421494e+00, ...,
         2.68527341e+00, -3.88384432e-01,  2.09120219e-04],
       [ 5.38184881e-01,  6.75068915e-01, -1.07786560e+00, ...,
         2.67647243e+00, -4.81758177e-01,  2.02324896e-04],
       [ 5.27404070e-01,  6.77456260e-01, -1.07713532e+00, ...,
         2.67746496e+00, -4.98745501e-01,  2.00102382e-04]])

In [ ]:

X, y = np.array(X), np.array(y)
# print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [ ]:

print(y_train[0:10])

[[0 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]]


In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1418, 10, 132) (1418, 3)
(355, 10, 132) (355, 3)


In [ ]:
input=(X.shape[1], X.shape[2])
input

(10, 132)

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
model.compile(optimizer="Adam", metrics=['accuracy'], loss='categorical_crossentropy')


In [ ]:
model.fit(X_train, y_train, epochs=16, batch_size=32, validation_data=(X_test, y_test))
model.save("model.h5")


Epoch 1/16
45/45 [==============================] - 15s 42ms/step - loss: 0.8575 - accuracy: 0.5402 - val_loss: 0.5822 - val_accuracy: 0.6873
Epoch 2/16
45/45 [==============================] - 1s 12ms/step - loss: 0.4737 - accuracy: 0.7687 - val_loss: 0.2957 - val_accuracy: 0.8930
Epoch 3/16
45/45 [==============================] - 1s 12ms/step - loss: 0.2212 - accuracy: 0.9323 - val_loss: 0.1631 - val_accuracy: 0.9408
Epoch 4/16
45/45 [==============================] - 1s 12ms/step - loss: 0.1482 - accuracy: 0.9513 - val_loss: 0.1705 - val_accuracy: 0.9408
Epoch 5/16
45/45 [==============================] - 1s 14ms/step - loss: 0.1687 - accuracy: 0.9464 - val_loss: 0.1002 - val_accuracy: 0.9775
Epoch 6/16
45/45 [==============================] - 1s 17ms/step - loss: 0.1166 - accuracy: 0.9612 - val_loss: 0.1021 - val_accuracy: 0.9606
Epoch 7/16
45/45 [==============================] - 1s 17ms/step - loss: 0.1034 - accuracy: 0.9647 - val_loss: 0.1224 - val_accuracy: 0.9577
Epoch 8/16
4

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0)
print(score)

[0.13620354235172272, 0.9633802771568298]
